In [ ]:
!pip install swifter
!pip install swifter[groupby]

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat Nov 19 14:21:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    49W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# set desired python version

# https://stackoverflow.com/questions/63168301/how-to-change-the-python-version-from-default-3-5-to-3-8-of-google-colab
# !sudo apt-get update -y
# !sudo apt-get install python3.8
# from IPython.display import clear_output 
# clear_output()
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

# Choose one of the given alternatives:
# !sudo update-alternatives --config python3

# Check the result
!python3 --version

# Attention: Install pip (... needed!)
# !sudo apt install python3-pip

Python 3.7.15


In [ ]:
# libs
import json, os

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')
wd = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'GT', 'bd4h_proj')
os.chdir(wd)
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/GT/bd4h_proj'

In [ ]:
# set creds from license file
license = 'license.json'
with open(license, 'r') as f:
  license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

license_keys['JSL_VERSION'], license_keys['PUBLIC_VERSION']

('4.2.2', '4.2.2')

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh

--2022-11-18 03:40:43--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2128 (2.1K) [text/plain]
Saving to: ‘jsl_colab_setup.sh.1’

jsl_colab_setup.sh. 100%[===================>]   2.08K  --.-KB/s    in 0.001s  

2022-11-18 03:40:44 (2.17 MB/s) - ‘jsl_colab_setup.sh.1’ saved [2128/2128]



In [ ]:
# os.path.join(wd, 'jsl_colab_setup.sh')
os.chdir('/content/')

In [ ]:
# -p is for pyspark (by default 3.1.1)
!bash /content/drive/MyDrive/GT/bd4h_proj/jsl_colab_setup.sh -p

/content/drive/MyDrive/GT/bd4h_proj/jsl_colab_setup.sh: option requires an argument -- p
setup Colab for PySpark 3.1.1 and Spark NLP 4.2.2
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,066 kB]
Hit:11 http://ppa.launchpad.net/deadsna

In [ ]:
os.chdir(wd)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

     |████████████████████████████████| 212.4 MB 87 kB/s 
     |████████████████████████████████| 95 kB 3.1 MB/s 
     |████████████████████████████████| 66 kB 6.4 MB/s 
     |████████████████████████████████| 1.6 MB 30.0 MB/s 


In [ ]:
import json
import os

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sparknlp_jsl
import sparknlp

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
driver_memory = f'{int(ram_gb) - 2}G'

params = {"spark.driver.memory":driver_memory,
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params, gpu=True)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.2
Spark NLP_JSL Version : 4.2.2


Named Entity Recognition

In [1]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

In [ ]:
df = spark.read.option("delimiter", "|") \
  .option("header", "true") \
  .csv("aggregate_note_events-pipe-header.csv")
  
df = df.select('SUBJECT_ID', df.PATIENT_DOC.alias("text"))
df.printSchema()

root
 |-- SUBJECT_ID: string (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
import pandas as pd


def get_chunk_results(light_result):
  df = pd.DataFrame(columns = ['sentence', 'begin', 'end', 'chunks', 'entity', 'confidence', 'model'])

  for i in light_result.keys():

    model_name = "_".join(i.split("_")[:-1])

    if (i == 'sentence') | (i == 'token'):
      
      continue
    
    if ("_chunks" not in i):
      
      continue

    elif len(light_result[i]) == 0:
      pass
      # print("\n", "*"*20, model_name, "Model Results", "*"*20)
      # print("No Result For This Model")


    else:

      sentences = []
      begins = []
      ends = []
      chunks = []
      entities = []
      confidences = []

      for n in (light_result[i]):

        sentences.append(n.metadata['sentence'])
        begins.append(n.begin)
        ends.append(n.end) 
        chunks.append(n.result)
        entities.append(n.metadata['entity'])
        confidences.append(n.metadata['confidence'])

        
      df_model = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences})
      df_model['model'] = model_name
      df = pd.concat([df, df_model])
      # print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
      # display(df)
  return df

In [ ]:
from tqdm.notebook import tqdm
import time

for i in tqdm(range(10)):
  time.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from tqdm.notebook import tqdm
# empty_df = spark.createDataFrame([['']]).toDF("text")
# pipelineModel = clinical_profiling_pipeline.fit(empty_df)
# result = pipelineModel.transform(df)

####
for index, row in tqdm(df.toPandas().iterrows()):
  full_result = clinical_profiling_pipeline.fullAnnotate(row['text'])[0]
  df_chunks = get_chunk_results(full_result)
  ner_file = os.path.join('ner', f'pat-{row["SUBJECT_ID"]}-index-{index}.csv')
  df_chunks.to_csv(ner_file)
#   break
# full_result.head()
# df_chunks = get_chunk_results(full_result)
# df_chunks.count()
# print(full_result)
# result = clinical_profiling_pipeline.model.transform(df)
# result_pd = result.toPandas()
# result_pd.to_csv('ner_pre.csv')
# result.toJSON('ner_pre.json')
# result.to_avro('ner_pre.avro')
# result.limit(1).show()
# print(type(result))
# result.rdd.saveAsPickleFile('ner.pickle')

# .selectExpr("explode(pos)")

####
# result.write.parquet("ner.parquet")

# result.write.option("header", True).csv('ner_pre.csv')

0it [00:00, ?it/s]

In [ ]:
# testdf = df.select('PATIENT_DOC').limit(1)
# testdf.write \
#    .parquet("test_write.parquet")

In [ ]:
# text = df.select('PATIENT_DOC').take(1)#['PATIENT_DOC']
text = df.first()['PATIENT_DOC']
# for i, t in enumerate(text):
#   print(t)
  # if i==5:
    # break
text
len(text) / 434

131.44930875576037

In [ ]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''
len(text)

434

In [2]:
# %%time


test_df = spark.createDataFrame([[text]]).toDF("text")
test_result = clinical_profiling_pipeline.model.transform(test_df)
test_result.show()
# .selectExpr("explode(pos)")
# clinical_result = clinical_profiling_pipeline.annotate(text)
# clinical_result.keys()

Entity Resolution - UMLS

https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb#scrollTo=759IAMuoFrFZ

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', hcc=False, aux_label=False):
    
    full_light_result = lp.fullAnnotate(text)

    # chunks = []
    codes = []
    # begin = []
    # end = []
    # resolutions=[]
    # all_distances =[]
    # all_codes=[]
    # all_cosines = []
    # all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        # begin.append(chunk.begin)
        # end.append(chunk.end)
        # chunks.append(chunk.result)
        codes.append(code.result) 
        # all_codes.append(code.metadata['all_k_results'].split(':::'))
        # resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        # all_distances.append(code.metadata['all_k_distances'].split(':::'))
        # all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        
    #     if hcc:
    #         try:
    #             all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
    #         except:
    #             all_k_aux_labels.append([])
                
    #     elif aux_label:
    #         try:
    #             all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
    #         except:
    #             all_k_aux_labels.append([])
    #     else:
    #         all_k_aux_labels.append([])

    # df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes,'all_codes':all_codes, 
    #                    'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    # if hcc:

    #     df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
    #     df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
    #     df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])
    
    # elif aux_label:

    #     df['gt'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[0] for i in x])
    #     df['concept'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[1] for i in x])
    #     df['aux'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[2] for i in x])

    # df = df.drop(['all_k_aux_labels'], axis=1)
    
    # return df
    return codes


def get_code (lp, text, vocab='umls_code'):
    
    full_light_result = lp.fullAnnotate(text)
    codes = []

    for code in full_light_result[0][vocab]:
        codes.append(code.result) 
    return codes

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
umls_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        umls_resolver])

umls_lp = LightPipeline(umls_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
[OK!]


In [ ]:
# !pip install -U pandas
!pip show pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, pytz, python-dateutil
Required-by: xarray, vega-datasets, swifter, statsmodels, spark-nlp-display, sklearn-pandas, seaborn, pymc, prophet, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fastai, db-dtypes, datascience, cufflinks, cmdstanpy, arviz, altair


In [31]:
from tqdm.notebook import tqdm
import pandas as pd
import swifter

for file in tqdm(os.listdir(os.path.join(os.getcwd(), 'ner'))):
  ner_df = pd.read_csv(os.path.join('ner', file))
  ner_df['umls'] = ner_df['chunks'].swifter \
                .apply(lambda c: get_code (umls_lp, c))
  ner_df.to_csv(os.path.join('er', file))
  # break
ner_df.head()
# ner_df['umls'] = ner_df['chunks'] \
#                 .apply(lambda c: get_codes (umls_lp, c, 
#                                             vocab='umls_code', aux_label=True))
                
# ner_df.head()



  0%|          | 0/14 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/37697 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/21131 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15572 [00:00<?, ?it/s]

TypeError: ignored